In [1]:
import torch
import torch.nn as nn
import pandas as pd

class ScoreModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.net(x)

# Instantiate and load weights
model = ScoreModel()
model.load_state_dict(torch.load("../models/score_model.pth", weights_only=True))
model.eval()

ScoreModel(
  (net): Sequential(
    (0): Linear(in_features=8, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [2]:
def predict_score(budget_weight, safety_weight, youth_weight, centrality_weight,
                  norm_rent, norm_safety, norm_youth, norm_centrality):
    with torch.no_grad():
        x = torch.tensor([[budget_weight, safety_weight, youth_weight, centrality_weight,
                           norm_rent, norm_safety, norm_youth, norm_centrality]], dtype=torch.float32)
        return model(x).item()


In [3]:
import pandas as pd

df = pd.read_csv("../data/clean/borough_features.csv")

def recommend_top_n_boroughs(budget_weight, safety_weight, youth_weight, centrality_weight, top_n=5):
    recommendations = []

    for _, row in df.iterrows():
        borough = row["borough"]
        norm_rent = row["norm_rent"]
        norm_safety = row["norm_safety"]
        norm_youth = row["norm_youth"]
        norm_centrality = row["norm_centrality"]

        score = predict_score(
            budget_weight, safety_weight, youth_weight, centrality_weight,
            norm_rent, norm_safety, norm_youth, norm_centrality
        )

        recommendations.append({
            "borough": borough,
            "norm_rent": norm_rent,
            "norm_safety": norm_safety,
            "norm_youth": norm_youth,
            "norm_centrality": norm_centrality,
            "score": score
        })

    recommendations.sort(key=lambda x: x["score"], reverse=True)
    return recommendations[:top_n]


In [4]:
# Test different user profiles
test_profiles = [
    {"label": "Budget-focused", "budget_weight": 1.0, "safety_weight": 0.0, "youth_weight": 0.0, "centrality_weight": 0.0},
    {"label": "Safety-focused", "budget_weight": 0.0, "safety_weight": 1.0, "youth_weight": 0.0, "centrality_weight": 0.0},
    {"label": "Urban enthusiast", "budget_weight": 0.0, "safety_weight": 0.0, "youth_weight": 0.0, "centrality_weight": 1.0},
    {"label": "Balanced", "budget_weight": 0.3, "safety_weight": 0.3, "youth_weight": 0.2, "centrality_weight": 0.2},
    {"label": "Luxury young urban", "budget_weight": 0.0, "safety_weight": 0.1, "youth_weight": 0.6, "centrality_weight": 0.3},
]

for profile in test_profiles:
    print(f"\nTop boroughs for {profile['label']}:")
    top = recommend_top_n_boroughs(
        budget_weight=profile["budget_weight"],
        safety_weight=profile["safety_weight"],
        youth_weight=profile["youth_weight"],
        centrality_weight=profile["centrality_weight"],
        top_n=5
    )
    for rec in top:
        print(f"{rec['borough']:25s} | score={rec['score']:.4f} | rent={rec['norm_rent']:.2f} | safety={rec['norm_safety']:.2f} | youth={rec['norm_youth']:.2f} | centrality={rec['norm_centrality']:.2f}")



Top boroughs for Budget-focused:
barking and dagenham      | score=0.9240 | rent=0.96 | safety=0.84 | youth=0.26 | centrality=0.00
redbridge                 | score=0.8751 | rent=0.94 | safety=0.81 | youth=0.25 | centrality=0.00
havering                  | score=0.8619 | rent=0.99 | safety=0.87 | youth=0.17 | centrality=0.00
bexley                    | score=0.8580 | rent=1.00 | safety=0.93 | youth=0.13 | centrality=0.00
croydon                   | score=0.8139 | rent=0.95 | safety=0.65 | youth=0.23 | centrality=0.00

Top boroughs for Safety-focused:
kingston upon thames      | score=1.0646 | rent=0.74 | safety=0.99 | youth=0.22 | centrality=0.00
richmond upon thames      | score=1.0494 | rent=0.56 | safety=1.00 | youth=0.00 | centrality=0.00
merton                    | score=1.0349 | rent=0.68 | safety=0.96 | youth=0.30 | centrality=0.00
sutton                    | score=0.9130 | rent=0.94 | safety=0.97 | youth=0.10 | centrality=0.00
harrow                    | score=0.9084 | rent=0.